In [1]:
import googlemaps
import pandas as pd
import os
from dotenv import load_dotenv
import re
import requests
from bs4 import BeautifulSoup

### Get the list of Starbucks stores across Hong Kong

In [2]:
webpage_response = requests.get("https://onlinestore.starbucks.com.hk/pages/starbucks-stores-in-hong-kong-%E6%98%9F%E5%B7%B4%E5%85%8B%E9%A6%99%E6%B8%AF%E5%88%86%E5%BA%97")
webpage = webpage_response.content
soup = BeautifulSoup(webpage, "html.parser")

In [3]:
# Get the list of starbucks in hong kong
short_list = []
long_list = []
for i in soup.find_all(['tr']):
    td_list = i.contents
    col1 = td_list[1]
    col2 = td_list[3]
    
    short = col1.contents[0]
    long = col2.contents[-1]
    
    if '\xa0' in short:
        short = short.replace("\xa0", "")
    if '\xa0' in long:
        long = long.replace("\xa0", "")

    short_list.append(short)
    long_list.append(long)

### Get the API Key

In [4]:
load_dotenv("api_key.env")
api_key = os.getenv('API_KEY')

gmaps = googlemaps.Client(key=api_key)

### Extract the relevant data

In [5]:
location_df = []
rating_df = []
num_rating_df = []
price_level_df = []
url_df = []
types_df = []
review5_df = []
coor_df = []

error_list = []

location_id = []
idnum = 0
for index in range(len(short_list)):    
    shop = short_list[index]
    adds = long_list[index]
    
    try:
        # Get the place id of each places
        place_name = f'starbucks {shop}'
        places_details = gmaps.places(place_name)
        places_details_results = places_details["results"][0]
        place_id = places_details_results['place_id']

        # Get the place information
        place = gmaps.place(place_id)
        place_result = place["result"]

        # Get relevant data for each place. 
        place_rating = place_result['rating']
        place_number_rating = place_result['user_ratings_total']
        place_price_level = place_result['price_level']
        place_url = place_result['url']
        place_types = place_result['types']

        place_reviews5 = place_result['reviews']
        
        coordinates = place_result["geometry"]["location"]

        # Store and saved in csv file
        location_df.append(place_name)
        rating_df.append(place_rating)
        num_rating_df.append(place_number_rating)
        price_level_df.append(place_price_level)
        url_df.append(place_url)
        types_df.append(place_types)
        review5_df.append(place_reviews5)
        coor_df.append(coordinates)
        
        location_id.append(f"id_{idnum}")

    except KeyError as e:
        try: 
            print(f"IndexError '{shop}', use the longer address to get the data", end="\n"*3)

            # Get the place id of each places
            place_name = f'starbucks {adds}'
            places_details = gmaps.places(place_name)
            places_details_results = places_details["results"][0]
            place_id = places_details_results['place_id']

            # Get the place information
            place = gmaps.place(place_id)
            place_result = place["result"]

            # Get relevant data for each place. 
            place_rating = place_result['rating']
            place_number_rating = place_result['user_ratings_total']
            place_price_level = place_result['price_level']
            place_url = place_result['url']
            place_types = place_result['types']

            place_reviews5 = place_result['reviews']
            
            coordinates = place_result["geometry"]["location"]

            # Store and saved in csv file
            location_df.append(place_name)
            rating_df.append(place_rating)
            num_rating_df.append(place_number_rating)
            price_level_df.append(place_price_level)
            url_df.append(place_url)
            types_df.append(place_types)
            review5_df.append(place_reviews5)
            coor_df.append(coordinates)
            
            location_id.append(f"id_{idnum}")
            
        except IndexError as e:
            print(f"IndexError still encountered, skipping to next iteration.", end="\n"*3)
            error_list.append(adds)
            continue
        except KeyError as e:
            print(f"KeyError still encountered, skipping to next iteration.", end="\n"*3)
            error_list.append(adds)
            continue        
    except IndexError as e:
        try: 
            print(f"IndexError '{shop}', use the longer address to get the data", end="\n"*3)
            
            # Get the place id of each places
            place_name = f'starbucks {adds}'
            places_details = gmaps.places(place_name)
            places_details_results = places_details["results"][0]
            place_id = places_details_results['place_id']

            # Get the place information
            place = gmaps.place(place_id)
            place_result = place["result"]

            # Get relevant data for each place. 
            place_rating = place_result['rating']
            place_number_rating = place_result['user_ratings_total']
            place_price_level = place_result['price_level']
            place_url = place_result['url']
            place_types = place_result['types']

            place_reviews5 = place_result['reviews']
            
            coordinates = place_result["geometry"]["location"]

            # Store and saved in csv file
            location_df.append(place_name)
            rating_df.append(place_rating)
            num_rating_df.append(place_number_rating)
            price_level_df.append(place_price_level)
            url_df.append(place_url)
            types_df.append(place_types)
            review5_df.append(place_reviews5)
            coor_df.append(coordinates)
            
            location_id.append(f"id_{idnum}")
            
        except IndexError as e:
            print(f"IndexError still encountered, skipping to next iteration.", end="\n"*3)
            error_list.append(adds)
            continue
        except KeyError as e:
            print(f"KeyError still encountered, skipping to next iteration.", end="\n"*3)
            error_list.append(adds)
            continue
            
    idnum+=1
# Save in csv file
print("Done Extraction")
data = {"location_id": location_id,"location":location_df,
        "rating":rating_df, "num_rating":num_rating_df,
        "price_level":price_level_df, "url":url_df,
        "types":types_df, "coordinates": coor_df,
        "top5reviews":review5_df}
pd.DataFrame(data).to_csv("googlemaps_info.csv", index=False)

IndexError 'Hong Kong University Student Union', use the longer address to get the data


IndexError '<span style="color: #000000;"><a href="mailto:Life@KCC" style="color: #000000; text-decoration: underline;">Life @ KCC</a></span>', use the longer address to get the data


IndexError 'HK Station', use the longer address to get the data


IndexError still encountered, skipping to next iteration.


IndexError 'Magnet Place', use the longer address to get the data


KeyError still encountered, skipping to next iteration.


IndexError 'Peak Galleria No.203', use the longer address to get the data


IndexError 'Sun Arcade B1', use the longer address to get the data


IndexError still encountered, skipping to next iteration.


IndexError '中國建設銀行中心 China Construction Bank Centre', use the longer address to get the data


KeyError still encountered, skipping to next iteration.


IndexError 'Un Chau ', use the longer address to get the data


IndexError still encountered, skipping to next iter

In [6]:
# following locations were ignored
error_list

[' Concession HOK 3a & b LAR Hong Kong Station',
 ' Shop 1, G/F, Magnet Place 77-81 Container Port Rd, Kwai Chung',
 ' B1/F, T Galleria, The Sun Arcade, 28 Canton Road, Tsimshatsui, KLN',
 ' Portion of G/F, China Construction Bank Centre, 18 Wang Chiu Road, Kowloon Bay, Kowloon',
 '九龍長沙灣元州街303號,元州商埸地下G01號舖 Shop G01, G/F, Un Chau Shopping Ctr,303 Un Chau St, Cheung Sha Wan, Kln',
 ' Shop No. R23B, 3/F, Choi Yuen Plaza,Sheung Shui, N.T., Hong Kong',
 ' Unit 7T107, 7/F, Departure Hall,Terminal 1, Hong Kong Intl Airport']